In [2]:
import pandas as pd
import re
import numpy as np

import os
import warnings
warnings.filterwarnings('ignore')
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

In [6]:
player_stats = pd.read_csv("/Users/ciaran/Documents/Projects/AFL/data/scored_player_stats.csv")
player_stats['Season'] = player_stats['Match_ID'].apply(lambda x: int(x[:4]))
player_stats['Player_Season'] = player_stats['Player'] + "_" + player_stats['Season'].astype(str)

player_stats['Score'] = player_stats['Behinds'] + player_stats['Goals']*6
player_stats['xScore_Diff'] = player_stats['Score'] - player_stats['xScore']

player_stats['Home_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Away_Team'] = player_stats['Match_ID'].apply(lambda x: x.split("_")[-1]).str.replace(r"([A-Z])", r" \1").str.strip()
player_stats['Opponent'] = np.where(player_stats['Team'] == player_stats['Home_Team'], player_stats['Away_Team'], player_stats['Home_Team'])

player_stats.head()

,Match_ID,Team,Player,Round_ID,AFL_API_Player_ID,Player_Type,playerId,Age,Height,Weight,Number,Kicking_Foot,State_Of_Origin,Draft_Year,Debut_Year,Recruited_From,Draft_Position,Draft_Type,Photo_URL,Date_Of_Birth,Percent_Played,Behinds,Bounces,Centre_Bounces_Attended,Centre_Clearances,Clangers,Defensive_Contest_Losses,Defensive_Contest_Loss_Percentage,Defensive_One_On_One_Contests,Contested_Marks,Contested_Possession_Rate,Contested_Possessions,Offensive_One_On_One_Contests,Offensive_Contest_Wins,Offensive_Contest_Win_Percentage,Defensive_Half_Pressure_Acts,Disposal_Efficiency,Disposals,AFL_Fantasy_Points,Effective_Disposals,Effective_Kicks,Inside_50_Ground_Ball_Gets,Frees_Against,Frees_For,Goal_Accuracy,Goal_Assists,Goals,Ground_Ball_Gets,Handballs,Hit_Outs,Hit_Outs_To_Advantage,Hit_Outs_To_Advantage_Rate,Hit_Out_Win_Percentage,Inside_50s,Intercept_Marks,Intercepts,Kick_Efficiency,Kick_Ins,Kick_Ins_Played_On,Kicks,Kick_To_Handball_Ratio,Marks,Marks_Inside_50,Marks_On_Lead,Metres_Gained,One_Percenters,Pressure_Acts,Player_Rating_Points,Rebound_50s,Ruck_Contests,Score_Involvements,Score_Launches,Shots_At_Goal,Spoils,Stoppage_Clearances,Tackles,Tackles_Inside_50,Clearances,Possessions,Turnovers,Uncontested_Possessions,AFLCA_Player_ID,Coaches_Votes,Position,Team_Status,Position_Sub_Group,Position_Group,Year,Brownlow_Votes,Season,xScore,xT_created,xT_denied,vaep_value,offensive_value,defensive_value,exp_vaep_value,exp_offensive_value,exp_defensive_value,xT_received,xT_prevented,vaep_value_received,exp_vaep_value_received,Player_Season,Score,xScore_Diff,Home_Team,Away_Team,Opponent
0,202101_BrisbaneLions_Sydney,Brisbane Lions,Brandon Starcevich,202101,Brandon_Starcevich,MEDIUM_DEFENDER,CD_I1002401,22,187,92,37,RIGHT,WA,2017.0,2018.0,Mt Lawley-Inglewood (WA)/Trinity Coll (WA)/Eas...,18.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1999-07-24,84.0,0.0,1.0,0.0,0.0,1.0,1.0,100.0,1.0,1.0,55.6,5.0,0.0,0.0,0.0,4.0,88.9,9.0,31.0,8.0,5.0,0.0,0.0,1.0,0.0,0.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,83.3,0.0,0.0,6.0,2.0,2.0,0.0,1.0,172.0,1.0,5.0,6.1,2.0,0.0,4.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,9.0,2.0,4.0,NaN,NaN,Half Back Flank Left,FINAL_TEAM,Half-Back,Back,2021.0,0.0,2021,0.000000,-0.235667,0.000000,0.711460,0.026457,0.685004,3.362637,0.585132,2.777505,-0.461340,0.190552,0.329877,2.328707,Brandon Starcevich_2021,0.0,0.000000,Brisbane Lions,Sydney,Sydney
1,202101_BrisbaneLions_Sydney,Brisbane Lions,Callum Ah Chee,202101,Callum_Ah_Chee,MEDIUM_DEFENDER,CD_I298437,24,183,79,4,RIGHT,WA,2015.0,2016.0,Kelmscott (WA)/South Fremantle (WAFL)/Gold Coast,8.0,nationalDraft,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1997-10-09,87.0,0.0,0.0,0.0,0.0,4.0,1.0,100.0,1.0,0.0,66.7,8.0,0.0,0.0,0.0,5.0,83.3,12.0,36.0,10.0,5.0,0.0,2.0,0.0,0.0,1.0,0.0,8.0,6.0,0.0,0.0,0.0,0.0,2.0,0.0,7.0,83.3,1.0,1.0,6.0,1.0,0.0,0.0,0.0,258.0,2.0,7.0,8.0,1.0,0.0,2.0,1.0,0.0,2.0,0.0,3.0,1.0,0.0,12.0,6.0,4.0,NaN,NaN,Interchange,FINAL_TEAM,Interchange,Interchange,2021.0,0.0,2021,0.000000,-0.075963,0.010892,0.291451,0.479986,-0.188535,1.318957,1.031427,0.287530,-0.487945,-0.039690,0.608221,3.269039,Callum Ah Chee_2021,0.0,0.000000,Brisbane Lions,Sydney,Sydney
2,202101_BrisbaneLions_Sydney,Brisbane Lions,Charlie Cameron,202101,Charlie_Cameron,MEDIUM_FORWARD,CD_I990609,27,180,75,23,RIGHT,WA,2014.0,2014.0,Newman Saints (WA)/Swan Districts (WAFL)/Adelaide,NaN,rookieElevation,https://s.afl.com.au/staticfile/AFL Tenant/AFL...,1994-07-05,77.0,0.0,0.0,9.0,3.0,0.0,0.0,0.0,0.0,0.0,50.0,5.0,0.0,0.0,0.0,3.0,70.0,10.0,39.0,7.0,4.0,0.0,0.0,2.0,0.0,1.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,3.0,0.0,1.0,57.1,0.0,0.0,7.0,2.3,2.0,1.0,2.0,207.0,1.0,9.0,10.1,0.0,0.0,4.0,1.0,0.0,0.0,0.0,1.0,0.0,3.0,10.0,0.0,5.0,NaN,NaN,Forward Pocket Right,FINAL_TEAM,Forward-Pocket,Forward,2021.0,0.0,2021,0.000000,0.460322,-0.166355,0.696752,0.681301,0.015451,6.034727,5.882581,0.152147,0.197894,-0.035491,0.859897,8.186274,Charlie Cameron_2021,0.0,0.000000,Brisbane Lions,Sydney,Sydney
3,202101_BrisbaneLions_Sydney,Br

In [8]:
match_summary = player_stats.groupby(['Match_ID', 'Team', 'Home_Team']).sum()[['Score', 'xScore']]
match_summary

Score  \
Match_ID                           Team             Home_Team                 
202101_BrisbaneLions_Sydney        Brisbane Lions   Brisbane Lions     90.0   
                                   Sydney           Brisbane Lions    122.0   
202101_Collingwood_WesternBulldogs Collingwood      Collingwood        52.0   
                                   Western Bulldogs Collingwood        66.0   
202101_Essendon_Hawthorn           Essendon         Essendon           89.0   
...                                                                     ...   
202323_Richmond_NorthMelbourne     Richmond         Richmond           94.0   
202323_StKilda_Geelong             Geelong          St Kilda           55.0   
                                   St Kilda         St Kilda           85.0   
202323_WesternBulldogs_WestCoast   West Coast       Western Bulldogs    0.0   
                                   Western Bulldogs Western Bulldogs    0.0   

                                                                          xScore  
Match_ID                           Team             Home_Team                     
202101_BrisbaneLions_Sydney        Brisbane Lions   Brisbane Lions     69.728083  
                                   Sydney           Brisbane Lions     92.468489  
202101_Collingwood_WesternBulldogs Collingwood      Collingwood        60.399395  
                                   Western Bulldogs Collingwood        89.958140  
202101_Essendon_Hawthorn           Essendon         Essendon           93.403011  
...                                                                          ...  
202323_Richmond_NorthMelbourne     Richmond         Richmond          100.451717  
202323_StKilda_Geelong             Geelong          St Kilda           57.614314  
                                   St Kilda         St Kilda          110.697849  
202323_WesternBulldogs_WestCoast   West Coast       Western Bulldogs   83.056271  
                                   Western Bulldogs Western Bulldogs   82.120013  

[1210 rows x 2 columns]